# Comparison Tests
for LiftOn, Liftoff, and miniprot
- on CHM13_MANE and CHM13_RefSeq

Navigate to the directory `/ccb/salz2/kh.chao/Lifton/results/` to access the LiftOn results. Within this directory, you’ll find subdirectories for `bee`, `rice`, `arabadop` (arabidopsis), `human_mane`, and `human_chess`
Each subdirectory contains a `score.txt` file with columns for “transcript ID,” “Liftoff protein sequence identity,” “miniprot protein sequence identity,” “LiftOn protein sequence identity,” “Category,” “Mutation types,” and “gene_locus.”
Here are three things we want to check first:
- LiftOn score is better than Liftoff & miniprot
- How many transcripts have the same score as Liftoff; how many transcripts have the same score as miniprot (this category means that LiftOn directly uses either Liftoff or miniprot’s CDSs)
- LiftOn score is worse than Liftoff & miniprot

For 1 and 3, you can use the automated igv.sh script to visualize the results on IGV. Specifically:
- For 1, we want to see how exactly LiftOn chains the CDSs and provides an improved annotation (and how many cases)
- For 3, examine why LiftOn may have inaccurately annotated. This helps us to improve the current version of LiftOn

The sequence identity dot plots and frequency plots are here: `/ccb/salz2/kh.chao/Lifton/results/arabadop/visualization/`.
In the identity dot plots, you can see there are still some dots that are on the bottom right panel. We want to fix those cases.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil
import re

OUTPUT_DIR = './output'

### collect scores from original score files
referencing kh's code from `ccb/salz2/kh.chao/Lifton/lifton/lifton_utils.py` -> `write_lifton_status()`

In [2]:
# def write_lifton_status(fw_score, transcript_id, transcript, lifton_status):
#     final_status = ";".join(lifton_status.status)
#     fw_score.write(f"{transcript_id}\t{lifton_status.liftoff}\t{lifton_status.miniprot}\t{lifton_status.lifton}\t{lifton_status.annotation}\t{final_status}\t{transcript.seqid}:{transcript.start}-{transcript.end}\n")

### generate stats from score comparisons

In [3]:
def read_score(score_file, verbose=False):
    # read the given species file into a dataframe
    column_names = ['transcript_id', 'liftoff_score', 'miniprot_score', 'lifton_score', 'category', 'mutation_type', 'gene_locus']
    df = pd.read_csv(score_file,  names=column_names, delimiter='\t')
    if verbose:
        print(f'Reading in {score_file}...')
        print(df.head())
    return df

In [4]:
'''!!!!!!WARNING: DELETING DIRECTORIES!!!!!!!'''
def delete_directory(directory_path):
    if os.path.exists(directory_path):
        # The directory exists, so delete it
        try:
            shutil.rmtree(directory_path)
            print(f"Directory '{directory_path}' deleted successfully.")
        except Exception as e:
            print(f"Error deleting directory '{directory_path}': {e}")
    else:
        print(f"Directory '{directory_path}' does not exist.")

In [5]:
def generate_stats(df, species, clean=False, verbose=False):
    # get filenames
    if verbose: 
        print(f'Collecting stats for {species}...')
    
    if clean:
        delete_directory(f'{OUTPUT_DIR}/{species}/')
        
    os.makedirs(f'{OUTPUT_DIR}/{species}/', exist_ok=True)
    lifton_better_filepath = f'{OUTPUT_DIR}/{species}/lifton_better.out'
    lifton_worse_filepath = f'{OUTPUT_DIR}/{species}/lifton_worse.out'
    stats_filepath = f'{OUTPUT_DIR}/{species}/score_stats.out'
    
    # collect statistics within dataframe
    for col1 in df.columns[1:4]:
        for col2 in df.columns[1:4]:
            if col1 != col2:
                is_greater = df[col1] > df[col2]
                is_equal = df[col1] == df[col2]

                df[f'{col1}>{col2}'] = is_greater
                df[f'{col1}={col2}'] = is_equal
    
    stats_counts = df.iloc[:, -12:].sum()

    if verbose:
        print(f'Collected stat counts:\n{stats_counts}\n')
    
    # write selected scenarios to file
    out = '-------------- SCENARIOS ------------\n'
    ## 1. lifton best
    lifton_best = df[df['lifton_score>liftoff_score'] & df['lifton_score>miniprot_score']].iloc[:, :-12]
    lifton_best.to_csv(lifton_better_filepath, index=False, header=False, sep='\t')
    out += f'LiftOn best\t\t\t\t\t\t{len(lifton_best)}\n'
    ## 2. liftoff best
    liftoff_best = df[df['liftoff_score>lifton_score'] & df['liftoff_score>miniprot_score']]
    out += f'Liftoff best\t\t\t\t\t{len(liftoff_best)}\n'
    ## 3. miniprot best
    miniprot_best = df[df['miniprot_score>lifton_score'] & df['miniprot_score>liftoff_score']]
    out += f'miniprot best\t\t\t\t\t{len(miniprot_best)}\n'
    ## 4. lifton < liftoff only
    lifton_worse_liftoff = df[df['liftoff_score>lifton_score'] & ~df['miniprot_score>lifton_score']].iloc[:, :-12]
    lifton_worse_liftoff.to_csv(lifton_worse_filepath, index=False, header=False, sep='\t')
    out += f'LiftOn worse than Liftoff only\t{len(lifton_worse_liftoff)}\n' 
    ## 5. lifton < miniprot only
    lifton_worse_miniprot = df[df['miniprot_score>lifton_score'] & ~df['liftoff_score>lifton_score']].iloc[:, :-12]
    lifton_worse_miniprot.to_csv(lifton_worse_filepath, mode='a', index=False, header=False, sep='\t')
    out += f'LiftOn worse than miniprot only\t{len(lifton_worse_miniprot)}\n' 
    ## 6. lifton worst
    lifton_worst = df[df['liftoff_score>lifton_score'] & df['miniprot_score>lifton_score']].iloc[:, :-12]
    lifton_worst.to_csv(lifton_worse_filepath, mode='a', index=False, header=False, sep='\t')
    out += f'LiftOn worst\t\t\t\t\t{len(lifton_worst)}\n' 
    ## 7. lifton == liftoff only
    lifton_eq_liftoff = df[df['lifton_score=liftoff_score'] & ~df['lifton_score=miniprot_score']]
    out += f'LiftOn equal to Liftoff only\t{len(lifton_eq_liftoff)}\n'
    ## 8. lifton == miniprot only 
    lifton_eq_miniprot = df[df['lifton_score=miniprot_score'] & ~df['lifton_score=liftoff_score']]
    out += f'LiftOn equal to miniprot only\t{len(lifton_eq_miniprot)}\n'
    ## 9. all equal
    all_eq = df[df['lifton_score=miniprot_score'] & df['lifton_score=liftoff_score']]
    out += f'All equal\t\t\t\t\t\t{len(all_eq)}\n'
    ## 10. total counted
    out += f'Total counts\t\t\t\t\t{len(df)}\n'

    # do all mutually exclusive cases, ranked
    out += '\n----------------- ALL ---------------\n'
    out += f"lifton=liftoff=miniprot\t{len(all_eq)}\n"
    out += f"lifton>liftoff=miniprot\t{len(df[df['lifton_score>liftoff_score'] & df['liftoff_score=miniprot_score']])}\n"
    out += f"lifton>liftoff>miniprot\t{len(df[df['lifton_score>liftoff_score'] & df['liftoff_score>miniprot_score']])}\n"
    out += f"lifton>miniprot>liftoff\t{len(df[df['lifton_score>miniprot_score'] & df['miniprot_score>liftoff_score']])}\n"
    out += f"lifton=liftoff>miniprot\t{len(df[df['lifton_score=liftoff_score'] & df['lifton_score>miniprot_score']])}\n"
    out += f"lifton=miniprot>liftoff\t{len(df[df['lifton_score=miniprot_score'] & df['lifton_score>liftoff_score']])}\n"
    out += f"liftoff>lifton>miniprot\t{len(df[df['liftoff_score>lifton_score'] & df['lifton_score>miniprot_score']])}\n"
    out += f"miniprot>lifton>liftoff\t{len(df[df['miniprot_score>lifton_score'] & df['lifton_score>liftoff_score']])}\n"
    out += f"liftoff>lifton=miniprot\t{len(df[df['liftoff_score>lifton_score'] & df['lifton_score=miniprot_score']])}\n"
    out += f"miniprot>lifton=liftoff\t{len(df[df['miniprot_score>lifton_score'] & df['lifton_score=liftoff_score']])}\n"
    out += f"liftoff=miniprot>lifton\t{len(df[df['liftoff_score=miniprot_score'] & df['liftoff_score>lifton_score']])}\n"
    out += f"liftoff>miniprot>lifton\t{len(df[df['liftoff_score>miniprot_score'] & df['miniprot_score>lifton_score']])}\n"
    out += f"miniprot>liftoff>lifton\t{len(df[df['miniprot_score>liftoff_score'] & df['liftoff_score>lifton_score']])}\n"
    out += f"all\t\t\t\t\t\t{len(df)}\n"

    if verbose: 
        print(out)
    
    # adding raw data to the file
    with open(stats_filepath, 'a') as f:
        f.write(out)
        f.write('\n----------------- RAW ---------------\n')
    stats_counts.to_csv(stats_filepath, mode='a', header=False, sep='\t')

    if verbose:
        print(f'Wrote stat results to: {stats_filepath}, {lifton_better_filepath}, {lifton_worse_filepath}\n\n')

In [6]:
def aggregate_stats(species_list, output_path, verbose=False):

    # read in each individual stats_file and generate an aggregate file
    if verbose:
        print(f'Aggregating stats for {list(species_list)}...')

    data_list = []
    for species in species_list:
        
        # define file
        filepath = f'{OUTPUT_DIR}/{species}/score_stats.out'
        if verbose:
            print(f'Parsing {filepath}...')
    
        # read the lines from the file that don't start with '-' or '\n'
        with open(filepath, 'r') as file:
            lines = [line.strip() for line in file.readlines() if not line.startswith('-') and line.strip()]
        data = '\t'.join(lines)

        # use regular expression to extract key-value pairs separated by tabs
        pattern = re.compile(r'([^\t]+)\t+([^\t]+)')
        matches_dict = dict(pattern.findall(data))

        # add the filename as a key in the dictionary
        matches_dict['species'] = species

        # Append the dictionary to the list
        data_list.append(matches_dict)
    
    # create a DataFrame from the list
    df = pd.DataFrame(data_list)
    df.set_index(df.columns[-1], inplace=True)
    df.to_csv(output_path)
    df.to_excel(os.path.splitext(output_path)[0]+'.xlsx')

    if verbose:
        print(df.head())
        print(f'Data compiled at {output_path}.')
    


In [7]:
'''========================= DRIVER ========================'''
# collecting the scores into an output file
aggregate_file = f'{OUTPUT_DIR}/agg_score_stats.csv'
SPECIES = ['bee', 'bee_test', 'rice', 'rice_test', 'arabadop', 'arabadop_test', 'human_mane', 'human_mane_test', 'human_chess', 'human_chess_test']

# for s in SPECIES:
#     df = read_score(f'./results/{s}/score.txt')
#     generate_stats(df, s, clean=True, verbose=True)

aggregate_stats(SPECIES, aggregate_file, verbose=True)



Aggregating stats for ['bee', 'bee_test', 'rice', 'rice_test', 'arabadop', 'arabadop_test', 'human_mane', 'human_mane_test', 'human_chess', 'human_chess_test']...
Parsing ./output/bee/score_stats.out...
Parsing ./output/bee_test/score_stats.out...
Parsing ./output/rice/score_stats.out...
Parsing ./output/rice_test/score_stats.out...
Parsing ./output/arabadop/score_stats.out...
Parsing ./output/arabadop_test/score_stats.out...
Parsing ./output/human_mane/score_stats.out...
Parsing ./output/human_mane_test/score_stats.out...
Parsing ./output/human_chess/score_stats.out...
Parsing ./output/human_chess_test/score_stats.out...
          LiftOn best Liftoff best miniprot best  \
species                                            
bee               548            3             0   
bee_test          565            3             0   
rice             1334            2            18   
rice_test        1334            2            18   
arabadop         3101           16            40   

     

### visualize results with igv viewer